<a href="https://colab.research.google.com/github/jimwhite/commentator_ai/blob/main/Transcript_to_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
%pip install -q stability-sdk youtube-transcript-api langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.2 MB/s eta 0:00:00


In [2]:
#@title Set up Google Drive for file storage
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    outputs_path = "/content/gdrive/MyDrive/Commentator_AI/Transcript_to_Video"
    !mkdir -p $outputs_path
except:
    outputs_path = "."
print(f"Files will be saved to {outputs_path}")

Mounted at /content/gdrive
Files will be saved to /content/gdrive/MyDrive/Commentator_AI/Transcript_to_Video


In [12]:
#@title Set YouTube URL
import re
def ytIdFromURL(url:str)->str:
   data = re.findall(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
   if data:
       return data[0]
   return None

YOUTUBE_URL = "https://www.youtube.com/watch?v=vPKp29Luryc" #@param {type:"string"}
video_id = ytIdFromURL(YOUTUBE_URL)
print(f'YouTube video ID = {video_id}')

YouTube video ID = vPKp29Luryc


In [19]:
#@title Get Video Transcript (JSON)
import json
import os
from youtube_transcript_api import YouTubeTranscriptApi

if not video_id:
  raise ValueError("video_id isn't set")

transcript = None
out_dir = os.path.join(outputs_path, video_id)
transcript_file_path = os.path.join(out_dir, 'transcript.json')
if os.path.exists(transcript_file_path):
  with open(transcript_file_path, 'r') as json_file:
    transcript = json.load(json_file)
    print(f'Read transcript from file: {transcript_file_path}')

if not transcript:
  transcript = YouTubeTranscriptApi.get_transcript(video_id)
  print('Got transcript from YouTube API')
  os.makedirs(out_dir, exist_ok=True)
  with open(transcript_file_path, 'w') as json_file:
    json.dump(transcript, json_file)
    print(f'Saved transcript to file: {transcript_file_path}')

transcript

Read transcript from file: /content/gdrive/MyDrive/Commentator_AI/Transcript_to_Video/vPKp29Luryc/transcript.json


[{'text': 'O say can you see,', 'start': 7.407, 'duration': 4.071},
 {'text': "by the dawn's early light,", 'start': 11.478, 'duration': 3.737},
 {'text': 'What so proudly we hailed', 'start': 15.215, 'duration': 3.837},
 {'text': "at the twilight's last gleaming?",
  'start': 19.052,
  'duration': 3.937},
 {'text': 'Whose broad stripes and bright stars,',
  'start': 22.989,
  'duration': 3.937},
 {'text': 'through the perilous fight,', 'start': 26.926, 'duration': 3.537},
 {'text': "O'er the ramparts we watched,", 'start': 30.463, 'duration': 3.737},
 {'text': 'were so gallantly streaming?', 'start': 34.2, 'duration': 4.071},
 {'text': 'And the rockets red glare,', 'start': 38.271, 'duration': 3.837},
 {'text': 'the bombs bursting in air,', 'start': 42.108, 'duration': 3.837},
 {'text': 'Gave proof through the night', 'start': 45.945, 'duration': 3.737},
 {'text': 'that our flag was still there.',
  'start': 49.682,
  'duration': 3.871},
 {'text': 'O say does that', 'start': 53.553, '

In [24]:
#@title Get Video Transcript (CSV)

# Using CSV files is a convenient way to integrate with LangChain (and LangFlow).
# Also is much more efficient in token usage so longer transcripts will work for
# any given LLM context token limit.

import csv
import os
from youtube_transcript_api import YouTubeTranscriptApi

if not video_id:
  raise ValueError("video_id isn't set")

transcript = []
out_dir = os.path.join(outputs_path, video_id)
transcript_file_path = os.path.join(out_dir, 'transcript.csv')
fieldnames = ['start', 'duration', 'text']
if os.path.exists(transcript_file_path):
  with open(transcript_file_path, 'r') as csv_file:
    reader = csv.DictReader(csv_file, fieldnames=fieldnames, quoting=csv.QUOTE_NONNUMERIC)
    next(reader)  # skip header
    for row in reader:
      transcript.append(row)
    print(f'Read transcript from file: {transcript_file_path}')

if not transcript:
  transcript = YouTubeTranscriptApi.get_transcript(video_id)
  print('Got transcript from YouTube API')
  os.makedirs(out_dir, exist_ok=True)
  with open(transcript_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames, quoting=csv.QUOTE_NONNUMERIC)
    writer.writeheader()
    for row in transcript:
      writer.writerow(row)
    print(f'Saved transcript to file: {transcript_file_path}')

transcript

Read transcript from file: /content/gdrive/MyDrive/Commentator_AI/Transcript_to_Video/vPKp29Luryc/transcript.csv


[{'start': 7.407, 'duration': 4.071, 'text': 'O say can you see,'},
 {'start': 11.478, 'duration': 3.737, 'text': "by the dawn's early light,"},
 {'start': 15.215, 'duration': 3.837, 'text': 'What so proudly we hailed'},
 {'start': 19.052,
  'duration': 3.937,
  'text': "at the twilight's last gleaming?"},
 {'start': 22.989,
  'duration': 3.937,
  'text': 'Whose broad stripes and bright stars,'},
 {'start': 26.926, 'duration': 3.537, 'text': 'through the perilous fight,'},
 {'start': 30.463, 'duration': 3.737, 'text': "O'er the ramparts we watched,"},
 {'start': 34.2, 'duration': 4.071, 'text': 'were so gallantly streaming?'},
 {'start': 38.271, 'duration': 3.837, 'text': 'And the rockets red glare,'},
 {'start': 42.108, 'duration': 3.837, 'text': 'the bombs bursting in air,'},
 {'start': 45.945, 'duration': 3.737, 'text': 'Gave proof through the night'},
 {'start': 49.682,
  'duration': 3.871,
  'text': 'that our flag was still there.'},
 {'start': 53.553, 'duration': 2.603, 'text': '

In [27]:
transcript_csv_text = ''
with open(transcript_file_path, 'r') as csv_file:
  transcript_csv_text = csv_file.read()

transcript_csv_text

'"start","duration","text"\n7.407,4.071,"O say can you see,"\n11.478,3.737,"by the dawn\'s early light,"\n15.215,3.837,"What so proudly we hailed"\n19.052,3.937,"at the twilight\'s last gleaming?"\n22.989,3.937,"Whose broad stripes and bright stars,"\n26.926,3.537,"through the perilous fight,"\n30.463,3.737,"O\'er the ramparts we watched,"\n34.2,4.071,"were so gallantly streaming?"\n38.271,3.837,"And the rockets red glare,"\n42.108,3.837,"the bombs bursting in air,"\n45.945,3.737,"Gave proof through the night"\n49.682,3.871,"that our flag was still there."\n53.553,2.603,"O say does that"\n56.156,6.573,"star spangled banner yet wave"\n62.729,4.871,"O\'er the land of the free,"\n67.6,7.608,"and the home of the brave."\n'

In [54]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from getpass import getpass

if not 'OPENAI_API_KEY' in os.environ:
  key = getpass('Enter your OpenAI API key: ')
  if key:
    os.environ['OPENAI_API_KEY'] = key

CHAT_MODEL = 'gpt-4-0613'  #@param {type:"string"}
llm = ChatOpenAI(temperature=0.3, model=CHAT_MODEL)
image_gen = ImageGeneratorTool()
tools = [Tool(name='VideoImageGenerator',
              func=image_gen.run,
              description="""Image generator for making videos.
              Arguments are 'start', 'duration', 'lyric', 'image description'""")]
mrkl = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)


In [71]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

image_description_csv_text = None
out_dir = os.path.join(outputs_path, video_id)
image_description_file_path = os.path.join(out_dir, 'image_descriptions.csv')
fieldnames = ['start', 'duration', 'text', 'description']
if os.path.exists(image_description_file_path):
  with open(image_description_file_path, 'r') as csv_file:
    image_description_csv_text = csv_file.read()
    print(f'Read image descriptions from file: {image_description_file_path}')

if not image_description_csv_text:
  chat = ChatOpenAI(temperature=0.3, model=CHAT_MODEL)
  print('ChatGPT working...')
  response = chat([HumanMessage(content=f"""You're a visual musical artist.
  Given the following lyrics choose the phrases that should be illustrated to make a timed music video for this song.
  Respond in CSV format with the columns 'start', 'duration', 'text' (for the transcription text), 'description' (for the image description).
  Keep in mind that each image description will be rendered separately so don't use any references between them.

  {transcript_csv_text}""")])
  print('Got image descriptions from ChatGPT')
  print(response.content)
  os.makedirs(out_dir, exist_ok=True)
  with open(image_description_file_path, 'w', newline='') as csv_file:
    csv_file.write(response.content)
    print(f'Saved image descriptions to file: {image_description_file_path}')

image_description_csv_text

Read image descriptions from file: /content/gdrive/MyDrive/Commentator_AI/Transcript_to_Video/vPKp29Luryc/image_descriptions.csv
ChatGPT working...
Got image descriptions from ChatGPT
"start","duration","text","description"
7.407,4.071,"O say can you see,","A person looking out into the distance."
11.478,3.737,"by the dawn's early light,","A sunrise over a landscape."
15.215,3.837,"What so proudly we hailed","A group of people waving flags."
19.052,3.937,"at the twilight's last gleaming?","A sunset with a flag in the foreground."
22.989,3.937,"Whose broad stripes and bright stars,","Close-up of an American flag."
26.926,3.537,"through the perilous fight,","A battle scene with soldiers."
30.463,3.737,"O'er the ramparts we watched,","People watching from a fortress wall."
34.2,4.071,"were so gallantly streaming?","The American flag waving in the wind."
38.271,3.837,"And the rockets red glare,","Rockets lighting up the night sky."
42.108,3.837,"the bombs bursting in air,","Explosions in t

''

In [ ]:
#@title Connect to the Stability API
import datetime
import getpass
import json
import os
import panel as pn
import param

from base64 import b64encode
from IPython import display
from tqdm import tqdm
from types import SimpleNamespace

from stability_sdk.api import Context
from stability_sdk.animation import AnimationArgs, Animator
from stability_sdk.utils import create_video_from_frames

# @markdown To get your API key visit https://dreamstudio.ai/account
STABILITY_HOST = "grpc.stability.ai:443" #@param {type:"string"}
STABILITY_KEY = getpass.getpass('Enter your API Key')

# Connect to Stability API
context = Context(STABILITY_HOST, STABILITY_KEY)

# Test the connection
context.get_user_info()
print("Connection successful!")